In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing as pp
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
one = get_df('restaurants_one')
two = get_df('restaurants_two')
three = get_df('restaurants_three')
four = get_df('restaurants_four')
top_rest = get_df('restaurants')


In [ ]:
all_price = pd.concat([one, two, three, four, top_rest], axis =0)

In [ ]:
all_price.info()

In [ ]:
all_price

In [ ]:
top_rest['id'] = pd.to_numeric(top_rest.id)
all_price['id'] = pd.to_numeric(all_price.id)

In [ ]:
# Count and list the amount of null values
null_count = top_rest.price_level.isnull().groupby([top_rest['city']]).sum().astype(int).reset_index(name='count')
null_count

In [ ]:
null_count.sort_values('count')

In [ ]:
def graph_all(df):
    i = 1
    k = 5
    while k <= 140:
        sample = df[(df.id >= i) & (df.id <= k)]
        g = sns.FacetGrid(sample, col = "city")
        g.map(plt.hist, "price_level")
        plt.show()
        i += 5
        k += 5

In [ ]:
graph_all(all_price)

In [ ]:
user2 = all_price[all_price['city'].isin(['Sydney', 'London', 'Dublin', 'Prague', 'Rome'])]


In [ ]:
g = sns.FacetGrid(user2, col = "city")
g.map(plt.hist, "price_level")
plt.show()

In [ ]:
plt.hist(all_price.price_level)

In [ ]:
def to_city(df):
    city_df = df.groupby(['country', 'city', 'id', 'price_level'])['name'].count().to_frame()
    price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
    price_level['avg_price'] = df.groupby(['country', 'city', 'id'])['price_level'].mean()

    
    return price_level

In [ ]:
food = to_city(all_price)
food

In [ ]:
food.reset_index(inplace=True)
food.drop(columns = ['avg_price'], inplace=True)


In [ ]:
food

In [ ]:
le = pp.LabelEncoder()
food['label_id'] = le.fit_transform(food.city)

In [ ]:
food

In [ ]:
list(le.inverse_transform(food.label_id))

In [ ]:
food_new = food[['label_id', 1.0, 2.0, 3.0, 4.0]].copy()
food_new

In [ ]:
food_new.sort_values('label_id', inplace=True)
food_new.set_index('label_id', inplace=True)


In [ ]:
food_new.fillna(0, inplace=True)

In [ ]:
normalized = pp.normalize(food_new)
normalized_city = pd.DataFrame(normalized)
normalized_city


In [ ]:
survey_food = pd.read_csv('/Users/tristannisbet/Documents/SM/survey_food_only.csv', index_col=0)
survey_food

In [ ]:
replace_map = {'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Often': 3, 'Always': 4}

df_food_replace = survey_food.replace(replace_map)

In [ ]:
df_food_replace

In [ ]:
df_food_replace.drop(columns=['nationality', 'age', 'gender'], inplace=True)

In [ ]:
normalized_user = pd.DataFrame(pp.normalize(df_food_replace))
normalized_user

In [ ]:
cosine_sim = pd.DataFrame(cosine_similarity(normalized_user, normalized_city))
cosine_sim

In [ ]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
top_10_city = find_similar_n(cosine_sim,10)
top_10_city

In [ ]:
user_1 = top_10_city.iloc[0, 0:5].values.tolist()
user_1

In [ ]:
for city in user_1:
    city2 = food[food.label_id == city]
    print(city2.city)

In [ ]:
food.iloc[food.index.get_level_values('label_id') == 80]

In [ ]:
all_food.iloc[all_food.index.get_level_values('id') == 81]

In [ ]:
all_food